In [34]:
import sys, math
from collections import namedtuple
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter
import pygame
import random
import lzma
import os
from tqdm import tqdm
import warnings

DATA_DIRECTORY = "data/2023_2/"
INPUT_FILE = 'KA050_processed_10cm_5h_20230614.pkl.xz'

def load_data(source_dir, input_file, scale = None, arena_dim = None):
    data = None
    with lzma.open(os.path.join(source_dir, input_file)) as file:
        data = pd.read_pickle(file)
    return data.iloc[::int(scale)] if scale else data


def process_data(data, arena_dim):
    data_len = len(data)
    arena_bb = find_bounding_box(data)
    origin_arena = calculate_circle(*arena_bb)

    translation, scale = circle_transformation(origin_arena, arena_dim)

    apply_transform_scale(data, translation, scale)

    return data

data = load_data(DATA_DIRECTORY, INPUT_FILE)

In [35]:
data[0]

,x,y
0,180.0,225.0
1,180.0,225.0
2,180.0,225.0
3,180.0,224.0
4,180.0,224.0
...,...,...
863998,522.0,418.0
863999,522.0,418.0
864000,522.0,418.0
864001,522.0,418.0


In [36]:
def calculate_pheromone_intensity_time_series_array_discrete(
    data: pd.DataFrame,
    grid_size: tuple = (900, 900),
    coarse_grid_size: tuple = (20, 20),
    bounding_box: int = 5,
    decay_rate: float = 0.01,
    deposition_rate: float = 0.1,
    diffusion_sigma: float = 1,
    scale: bool = True,
    frame_rate: int = 60,
    snapshot_interval_sec: float = 1.0,
    save_to_file: bool = False,
    output_file_path: str = "pheromone_time_series.npy"
) -> np.ndarray:
    """
    Calculate a time series of discretized pheromone intensity maps based on ant positions and return as a NumPy array.
    
    Parameters:
    - data (pd.DataFrame): DataFrame where each pair of columns represents x and y positions of an ant.
    - grid_size (tuple): Size of the original simulation grid (height, width).
    - coarse_grid_size (tuple): Size of the discretized grid (height, width).
    - bounding_box (int): Size of the bounding box around each ant in the coarse grid.
    - decay_rate (float): Decay rate of pheromones per time step.
    - deposition_rate (float): Amount of pheromone deposited by an ant.
    - diffusion_sigma (float): Standard deviation for Gaussian diffusion.
    - scale (bool): Whether to scale the pheromone maps between 0 and 1.
    - frame_rate (int): Number of frames per second in the video data.
    - snapshot_interval_sec (float): Interval in seconds at which to capture pheromone map snapshots.
    - save_to_file (bool): If True, saves the time series to a .npy file.
    - output_file_path (str): File path to save the NumPy array if save_to_file is True.
    
    Returns:
    - pheromone_time_series (np.ndarray): 3D array with shape (num_snapshots, coarse_height, coarse_width).
    """
    
    # Calculate discretization factors
    coarse_height, coarse_width = coarse_grid_size
    fine_height, fine_width = grid_size
    factor_y = fine_height / coarse_height
    factor_x = fine_width / coarse_width
    
    if not (fine_height % coarse_height == 0 and fine_width % coarse_width == 0):
        raise ValueError("Fine grid dimensions must be divisible by coarse grid dimensions.")
    
    # Initialize pheromone grid
    pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)
    
    # Initialize list to store pheromone snapshots
    pheromone_time_series = []
    
    # Calculate the number of frames between snapshots
    frames_per_snapshot = int(snapshot_interval_sec * frame_rate)
    if frames_per_snapshot < 1:
        frames_per_snapshot = 1  # Ensure at least every frame is captured if interval is very small
    
    # Determine the number of ants based on DataFrame columns
    if isinstance(data.columns, pd.MultiIndex):
        ant_indices = data.columns.get_level_values(0).unique()
        coordinate_levels = data.columns.get_level_values(1).unique()
        if 'x' not in coordinate_levels or 'y' not in coordinate_levels:
            raise ValueError("DataFrame must have 'x' and 'y' as subcolumn levels.")
        num_ants = len(ant_indices)
    else:
        # Assuming flat columns alternating between x and y
        num_ants = len(data.columns) // 2
    
    # Adjust bounding box from fine to coarse grid
    bounding_box_coarse = max(1, math.ceil(bounding_box / factor_x))
    
    # Iterate through each time step with progress tracking
    for idx, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing Frames"):
        # Decay pheromone
        pheromone_map *= (1 - decay_rate)
        
        # Iterate through each ant
        for ant in range(num_ants):
            if isinstance(data.columns, pd.MultiIndex):
                x_col = (ant, 'x')
                y_col = (ant, 'y')
            else:
                x_col = data.columns[ant*2]
                y_col = data.columns[ant*2 + 1]
            
            try:
                x_fine = float(row[x_col])
                y_fine = float(row[y_col])
            except (ValueError, TypeError, KeyError):
                # Handle missing or invalid data
                continue
            
            # Map fine coordinates to coarse grid
            x_coarse = int(x_fine // factor_x)
            y_coarse = int(y_fine // factor_y)
            
            # Ensure coordinates are within coarse grid bounds
            if not (0 <= x_coarse < coarse_width and 0 <= y_coarse < coarse_height):
                continue
            
            # Define bounding box in coarse grid
            half_box = bounding_box_coarse // 2
            x_start = max(x_coarse - half_box, 0)
            x_end = min(x_coarse + half_box + 1, coarse_width)
            y_start = max(y_coarse - half_box, 0)
            y_end = min(y_coarse + half_box + 1, coarse_height)
            
            # Deposit pheromone in the bounding box
            pheromone_map[y_start:y_end, x_start:x_end] += deposition_rate
        
        # Optional: Diffuse pheromones to simulate spreading
        pheromone_map = gaussian_filter(pheromone_map, sigma=diffusion_sigma)
        
        # Capture snapshot if it's the right frame
        if (idx + 1) % frames_per_snapshot == 0:
            snapshot = pheromone_map.copy()
            pheromone_time_series.append(snapshot)
    
    if scale:
        # Convert list to NumPy array for efficient scaling
        pheromone_time_series = np.array(pheromone_time_series, dtype=np.float32)
        global_max = pheromone_time_series.max()
        if global_max > 0:
            pheromone_time_series /= global_max
        pheromone_time_series = np.clip(pheromone_time_series, 0, 1)
    else:
        pheromone_time_series = np.array(pheromone_time_series, dtype=np.float32)
    
    if save_to_file:
        np.save(output_file_path, pheromone_time_series)
        print(f"Pheromone time series saved to '{output_file_path}'")
    
    return pheromone_time_series


In [37]:
def calculate_pheromone_intensity_time_series_array_discrete_save(
    data: pd.DataFrame,
    grid_size: tuple = (900, 900),
    coarse_grid_size: tuple = (20, 20),
    bounding_box: int = 5,
    decay_rate: float = 0.01,
    deposition_rate: float = 0.1,
    diffusion_sigma: float = 1,
    scale: bool = True,
    frame_rate: int = 60,
    snapshot_interval_sec: float = 1.0,
    save_to_file: bool = True,
    output_file_path: str = "pheromone_time_series_discrete.npy"
) -> None:
    """
    Calculate a time series of discretized pheromone intensity maps based on ant positions and save as a NumPy array file.
    
    Parameters:
    - data (pd.DataFrame): DataFrame where each pair of columns represents x and y positions of an ant.
    - grid_size (tuple): Size of the original simulation grid (height, width).
    - coarse_grid_size (tuple): Size of the discretized grid (height, width).
    - bounding_box (int): Size of the bounding box around each ant in the coarse grid.
    - decay_rate (float): Decay rate of pheromones per time step.
    - deposition_rate (float): Amount of pheromone deposited by an ant.
    - diffusion_sigma (float): Standard deviation for Gaussian diffusion.
    - scale (bool): Whether to scale the pheromone maps between 0 and 1.
    - frame_rate (int): Number of frames per second in the video data.
    - snapshot_interval_sec (float): Interval in seconds at which to capture pheromone map snapshots.
    - save_to_file (bool): If True, saves the time series to a .npy file.
    - output_file_path (str): File path to save the NumPy array if save_to_file is True.
    
    Returns:
    - None
    """
    
    if save_to_file:
        # Calculate discretization factors
        coarse_height, coarse_width = coarse_grid_size
        fine_height, fine_width = grid_size
        factor_y = fine_height / coarse_height
        factor_x = fine_width / coarse_width
        
        if not (fine_height % coarse_height == 0 and fine_width % coarse_width == 0):
            raise ValueError("Fine grid dimensions must be divisible by coarse grid dimensions.")
        
        # Calculate frames per snapshot and number of snapshots
        frames_per_snapshot = int(snapshot_interval_sec * frame_rate)
        if frames_per_snapshot < 1:
            frames_per_snapshot = 1  # Ensure at least every frame is captured if interval is very small
        
        num_frames = data.shape[0]
        num_snapshots = num_frames // frames_per_snapshot
        
        # Initialize a memory-mapped file for efficient incremental writing
        pheromone_memmap = np.memmap(
            output_file_path,
            dtype='float32',
            mode='w+',
            shape=(num_snapshots, coarse_height, coarse_width)
        )
    else:
        pheromone_time_series = []
    
    # Determine the number of ants based on DataFrame columns
    if isinstance(data.columns, pd.MultiIndex):
        ant_indices = data.columns.get_level_values(0).unique()
        coordinate_levels = data.columns.get_level_values(1).unique()
        if 'x' not in coordinate_levels or 'y' not in coordinate_levels:
            raise ValueError("DataFrame must have 'x' and 'y' as subcolumn levels.")
        num_ants = len(ant_indices)
    else:
        # Assuming flat columns alternating between x and y
        num_ants = len(data.columns) // 2
    
    # Adjust bounding box from fine to coarse grid
    bounding_box_coarse = max(1, math.ceil(bounding_box / factor_x))
    
    # Initialize pheromone grid
    pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)
    
    # Initialize global maximum for scaling if required
    if scale:
        global_max = 1e-8  # Start with a small value to avoid division by zero
    
    # Initialize snapshot counter
    snapshot_counter = 0
    
    # Iterate through each time step with progress tracking
    with tqdm(total=num_frames, desc="Processing Frames") as pbar:
        for idx, row in data.iterrows():
            # Decay pheromone
            pheromone_map *= (1 - decay_rate)
            
            # Iterate through each ant
            for ant in range(num_ants):
                if isinstance(data.columns, pd.MultiIndex):
                    x_col = (ant, 'x')
                    y_col = (ant, 'y')
                else:
                    x_col = data.columns[ant*2]
                    y_col = data.columns[ant*2 + 1]
                
                try:
                    x_fine = float(row[x_col])
                    y_fine = float(row[y_col])
                except (ValueError, TypeError, KeyError):
                    # Handle missing or invalid data by skipping this ant
                    continue
                
                # Check for NaN values and skip if any are found
                if np.isnan(x_fine) or np.isnan(y_fine):
                    continue  # Skip this ant for the current frame
                
                # Map fine coordinates to coarse grid
                x_coarse = int(x_fine // factor_x)
                y_coarse = int(y_fine // factor_y)
                
                # Ensure coordinates are within coarse grid bounds
                if not (0 <= x_coarse < coarse_width and 0 <= y_coarse < coarse_height):
                    continue  # Skip if out of bounds
                
                # Define bounding box in coarse grid
                half_box = bounding_box_coarse // 2
                x_start = max(x_coarse - half_box, 0)
                x_end = min(x_coarse + half_box + 1, coarse_width)
                y_start = max(y_coarse - half_box, 0)
                y_end = min(y_coarse + half_box + 1, coarse_height)
                
                # Deposit pheromone in the bounding box
                pheromone_map[y_start:y_end, x_start:x_end] += deposition_rate
            
            # Optional: Diffuse pheromones to simulate spreading
            pheromone_map = gaussian_filter(pheromone_map, sigma=diffusion_sigma)
            
            # Capture snapshot if it's the right frame
            if (idx + 1) % frames_per_snapshot == 0:
                snapshot = pheromone_map.copy()
                
                if scale:
                    # Update global maximum
                    current_max = snapshot.max()
                    if current_max > global_max:
                        global_max = current_max
                    # Scale the snapshot
                    snapshot /= global_max
                    snapshot = np.clip(snapshot, 0, 1)
                
                if save_to_file:
                    pheromone_memmap[snapshot_counter] = snapshot
                    snapshot_counter += 1
                else:
                    pheromone_time_series.append(snapshot)
            
            pbar.update(1)
    
    if save_to_file:
        # Flush changes to disk
        pheromone_memmap.flush()
        print(f"Pheromone time series saved to '{output_file_path}' as a NumPy memmap file.")
    else:
        if scale:
            # Convert list to NumPy array for efficient scaling
            pheromone_time_series = np.array(pheromone_time_series, dtype=np.float32)
            pheromone_time_series /= pheromone_time_series.max()
            pheromone_time_series = np.clip(pheromone_time_series, 0, 1)
        else:
            pheromone_time_series = np.array(pheromone_time_series, dtype=np.float32)
        return pheromone_time_series


In [38]:
def calculate_pheromone_intensity_time_series_save_numpy(
    data: pd.DataFrame,
    grid_size: tuple = (900, 900),
    coarse_grid_size: tuple = (20, 20),
    bounding_box: int = 5,
    decay_rate: float = 0.01,
    deposition_rate: float = 0.1,
    diffusion_sigma: float = 1,
    scale: bool = True,
    frame_rate: int = 60,
    snapshot_interval_sec: float = 1.0,
    save_to_file: bool = True,
    output_file_path: str = "pheromone_time_series_discrete.npy"
) -> None:
    """
    Save pheromone time series as a standard .npy file using numpy.save().
    """
    if save_to_file:
        coarse_height, coarse_width = coarse_grid_size
        fine_height, fine_width = grid_size
        factor_y = fine_height / coarse_height
        factor_x = fine_width / coarse_width
        
        if not (fine_height % coarse_height == 0 and fine_width % coarse_width == 0):
            raise ValueError("Fine grid dimensions must be divisible by coarse grid dimensions.")
        
        frames_per_snapshot = int(snapshot_interval_sec * frame_rate)
        frames_per_snapshot = max(frames_per_snapshot, 1)
        
        num_frames = data.shape[0]
        num_snapshots = num_frames // frames_per_snapshot
        
        # Preallocate the pheromone time series array
        pheromone_time_series = np.zeros((num_snapshots, coarse_height, coarse_width), dtype='float32')
        
        # Determine the number of ants
        if isinstance(data.columns, pd.MultiIndex):
            ant_indices = data.columns.get_level_values(0).unique()
            coordinate_levels = data.columns.get_level_values(1).unique()
            if 'x' not in coordinate_levels or 'y' not in coordinate_levels:
                raise ValueError("DataFrame must have 'x' and 'y' as subcolumn levels.")
            num_ants = len(ant_indices)
        else:
            num_ants = len(data.columns) // 2
        
        bounding_box_coarse = max(1, math.ceil(bounding_box / factor_x))
        pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)
        
        global_max = 1e-8 if scale else None
        snapshot_counter = 0
        
        with tqdm(total=num_frames, desc="Processing Frames") as pbar:
            for idx, row in data.iterrows():
                pheromone_map *= (1 - decay_rate)
                
                for ant in range(num_ants):
                    if isinstance(data.columns, pd.MultiIndex):
                        x_col = (ant, 'x')
                        y_col = (ant, 'y')
                    else:
                        x_col = data.columns[ant*2]
                        y_col = data.columns[ant*2 + 1]
                    
                    try:
                        x_fine = float(row[x_col])
                        y_fine = float(row[y_col])
                    except (ValueError, TypeError, KeyError):
                        continue
                    
                    if np.isnan(x_fine) or np.isnan(y_fine):
                        continue
                    
                    x_coarse = int(x_fine // factor_x)
                    y_coarse = int(y_fine // factor_y)
                    
                    if not (0 <= x_coarse < coarse_width and 0 <= y_coarse < coarse_height):
                        continue
                    
                    half_box = bounding_box_coarse // 2
                    x_start = max(x_coarse - half_box, 0)
                    x_end = min(x_coarse + half_box + 1, coarse_width)
                    y_start = max(y_coarse - half_box, 0)
                    y_end = min(y_coarse + half_box + 1, coarse_height)
                    
                    pheromone_map[y_start:y_end, x_start:x_end] += deposition_rate
                
                pheromone_map = gaussian_filter(pheromone_map, sigma=diffusion_sigma)
                
                if (idx + 1) % frames_per_snapshot == 0:
                    snapshot = pheromone_map.copy()
                    
                    if scale:
                        current_max = snapshot.max()
                        if current_max > global_max:
                            global_max = current_max
                        snapshot /= global_max
                        snapshot = np.clip(snapshot, 0, 1)
                    
                    pheromone_time_series[snapshot_counter] = snapshot
                    snapshot_counter += 1
                
                pbar.update(1)
        
        # Save the pheromone time series using numpy.save()
        np.save(output_file_path, pheromone_time_series)
        print(f"Pheromone time series saved to '{output_file_path}' as a .npy file.")


In [39]:
import h5py
from typing import Tuple
import warnings

def calculate_pheromone_intensity_time_series_save_hdf5(
    data: pd.DataFrame,
    grid_size: Tuple[int, int] = (900, 900),
    coarse_grid_size: Tuple[int, int] = (20, 20),
    bounding_box: int = 5,
    decay_rate: float = 0.01,
    deposition_rate: float = 0.1,
    diffusion_sigma: float = 1,
    scale: bool = True,
    frame_rate: int = 60,
    snapshot_interval_sec: float = 1.0,
    save_to_file: bool = True,
    output_file_path: str = "pheromone_time_series_discrete.h5"
) -> None:
    """
    Save pheromone time series as an HDF5 file using h5py with global scaling.
    """
    if not save_to_file:
        return

    coarse_height, coarse_width = coarse_grid_size
    fine_height, fine_width = grid_size
    factor_y = fine_height / coarse_height
    factor_x = fine_width / coarse_width

    if not (fine_height % coarse_height == 0 and fine_width % coarse_width == 0):
        raise ValueError("Fine grid dimensions must be divisible by coarse grid dimensions.")

    frames_per_snapshot = int(snapshot_interval_sec * frame_rate)
    frames_per_snapshot = max(frames_per_snapshot, 1)

    num_frames = data.shape[0]
    num_snapshots = num_frames // frames_per_snapshot

    # Determine the number of ants
    if isinstance(data.columns, pd.MultiIndex):
        ant_indices = data.columns.get_level_values(0).unique()
        coordinate_levels = data.columns.get_level_values(1).unique()
        if 'x' not in coordinate_levels or 'y' not in coordinate_levels:
            raise ValueError("DataFrame must have 'x' and 'y' as subcolumn levels.")
        num_ants = len(ant_indices)
    else:
        num_ants = len(data.columns) // 2

    bounding_box_coarse = max(1, math.ceil(bounding_box / factor_x))

    # **First Pass: Determine global_max**
    print("First Pass: Determining global maximum pheromone intensity...")
    pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)
    global_max = 1e-8  # Initialize with a small number to avoid division by zero

    with tqdm(total=num_frames, desc="First Pass") as pbar:
        for idx, row in data.iterrows():
            # Apply decay
            pheromone_map *= (1 - decay_rate)

            # Update pheromone map with ant deposits
            for ant in range(num_ants):
                if isinstance(data.columns, pd.MultiIndex):
                    x_col = (ant, 'x')
                    y_col = (ant, 'y')
                else:
                    x_col = data.columns[ant*2]
                    y_col = data.columns[ant*2 + 1]

                try:
                    x_fine = float(row[x_col])
                    y_fine = float(row[y_col])
                except (ValueError, TypeError, KeyError):
                    continue

                if np.isnan(x_fine) or np.isnan(y_fine):
                    continue

                x_coarse = int(x_fine // factor_x)
                y_coarse = int(y_fine // factor_y)

                if not (0 <= x_coarse < coarse_width and 0 <= y_coarse < coarse_height):
                    continue

                half_box = bounding_box_coarse // 2
                x_start = max(x_coarse - half_box, 0)
                x_end = min(x_coarse + half_box + 1, coarse_width)
                y_start = max(y_coarse - half_box, 0)
                y_end = min(y_coarse + half_box + 1, coarse_height)

                pheromone_map[y_start:y_end, x_start:x_end] += deposition_rate

            # Optional: Apply diffusion
            # pheromone_map = gaussian_filter(pheromone_map, sigma=diffusion_sigma)

            # Update global_max
            current_max = pheromone_map.max()
            if current_max > global_max:
                global_max = current_max

            pbar.update(1)

    print(f"Global maximum pheromone intensity determined: {global_max}")

    # **Second Pass: Save scaled snapshots**
    print("Second Pass: Saving scaled pheromone snapshots...")
    # Reset pheromone_map
    pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)

    with h5py.File(output_file_path, 'w') as h5f:
        # Create dataset with extendable dimensions
        dset = h5f.create_dataset(
            "pheromone_time_series",
            shape=(num_snapshots, coarse_height, coarse_width),
            maxshape=(num_snapshots, coarse_height, coarse_width),
            dtype='float32'
        )

        snapshot_counter = 0

        with tqdm(total=num_frames, desc="Second Pass") as pbar:
            for idx, row in data.iterrows():
                # Apply decay
                pheromone_map *= (1 - decay_rate)

                # Update pheromone map with ant deposits
                for ant in range(num_ants):
                    if isinstance(data.columns, pd.MultiIndex):
                        x_col = (ant, 'x')
                        y_col = (ant, 'y')
                    else:
                        x_col = data.columns[ant*2]
                        y_col = data.columns[ant*2 + 1]

                    try:
                        x_fine = float(row[x_col])
                        y_fine = float(row[y_col])
                    except (ValueError, TypeError, KeyError):
                        continue

                    if np.isnan(x_fine) or np.isnan(y_fine):
                        continue

                    x_coarse = int(x_fine // factor_x)
                    y_coarse = int(y_fine // factor_y)

                    if not (0 <= x_coarse < coarse_width and 0 <= y_coarse < coarse_height):
                        continue

                    half_box = bounding_box_coarse // 2
                    x_start = max(x_coarse - half_box, 0)
                    x_end = min(x_coarse + half_box + 1, coarse_width)
                    y_start = max(y_coarse - half_box, 0)
                    y_end = min(y_coarse + half_box + 1, coarse_height)

                    pheromone_map[y_start:y_end, x_start:x_end] += deposition_rate

                # Optional: Apply diffusion
                # pheromone_map = gaussian_filter(pheromone_map, sigma=diffusion_sigma)

                # Check if it's time to take a snapshot
                if (idx + 1) % frames_per_snapshot == 0:
                    snapshot = pheromone_map.copy()

                    if scale:
                        snapshot /= global_max
                        snapshot = np.clip(snapshot, 0, 1)

                    dset[snapshot_counter] = snapshot
                    snapshot_counter += 1

                pbar.update(1)

    print(f"Pheromone time series saved to '{output_file_path}' as an HDF5 file.")


In [40]:
# # output_file = "pheromone_time_series_discrete.npy"
# output_file = "pheromone_time_series_discrete.h5"
# calculate_pheromone_intensity_time_series_save_hdf5_optimized(
#     data=data,
#     grid_size=(900, 900),
#     coarse_grid_size=(50, 50),  # Discretized to 20x20 grid
#     bounding_box=5,             # Bounding box in coarse grid
#     decay_rate=0.01,
#     deposition_rate=0.1,
#     diffusion_sigma=1,
#     scale=True,
#     frame_rate=60,
#     snapshot_interval_sec=1.0,  # Capture every 1 second
#     save_to_file=True,
#     output_file_path=os.path.join(DATA_DIRECTORY, output_file)
# )

In [41]:
def inspect_ant_positions(data: pd.DataFrame, grid_size: tuple):
    if isinstance(data.columns, pd.MultiIndex):
        ant_indices = data.columns.get_level_values(0).unique()
        x_cols = [col for col in data.columns if col[1] == 'x']
        y_cols = [col for col in data.columns if col[1] == 'y']
        x_positions = data[x_cols].values  # Shape: (num_frames, num_ants)
        y_positions = data[y_cols].values
        num_ants = len(ant_indices)
    else:
        raise ValueError("DataFrame does not have MultiIndex columns.")

    print(f"Number of Ants: {num_ants}")
    print(f"Shape of X Positions: {x_positions.shape}")
    print(f"Shape of Y Positions: {y_positions.shape}")

    # Check for NaNs
    num_nan_x = np.isnan(x_positions).sum()
    num_nan_y = np.isnan(y_positions).sum()
    print(f"Number of NaNs in X Positions: {num_nan_x}")
    print(f"Number of NaNs in Y Positions: {num_nan_y}")

    # Check positions within grid bounds
    fine_height, fine_width = grid_size
    x_within = (x_positions >= 0) & (x_positions < fine_width)
    y_within = (y_positions >= 0) & (y_positions < fine_height)
    total_within = np.sum(x_within & y_within)
    total_positions = x_positions.size
    print(f"Positions Within Bounds: {total_within}/{total_positions} ({(total_within / total_positions) * 100:.2f}%)")


In [42]:
inspect_ant_positions(data, grid_size=(900, 900))

Number of Ants: 57
Shape of X Positions: (864003, 57)
Shape of Y Positions: (864003, 57)
Number of NaNs in X Positions: 6082642
Number of NaNs in Y Positions: 6082642
Positions Within Bounds: 43165529/49248171 (87.65%)


In [43]:
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import math

def sigmoid_normalization(snapshot, k=0.1, x0=None):
    """
    Applies a sigmoid function to normalize the snapshot data between 0 and 1.

    Parameters:
    - snapshot (np.ndarray): 2D array of pheromone intensities.
    - k (float): Steepness of the sigmoid curve.
    - x0 (float): Midpoint of the sigmoid function. If None, set to half of the max value.

    Returns:
    - np.ndarray: Normalized snapshot.
    """
    if x0 is None:
        x0 = np.max(snapshot) / 2
    return 1 / (1 + np.exp(-k * (snapshot - x0)))


def calculate_pheromone_intensity_two_pass(
    data: pd.DataFrame,
    grid_size: tuple = (900, 900),
    coarse_grid_size: tuple = (20, 20),
    bounding_box: int = 5,
    decay_rate: float = 0.01,
    deposition_rate: float = 0.1,
    scale: bool = True,
    frame_rate: int = 60,
    snapshot_interval_sec: float = 1.0,
    save_to_file: bool = True,
    output_file_path: str = "pheromone_time_series_discrete_debug.h5"
) -> None:
    if not save_to_file:
        return

    coarse_height, coarse_width = coarse_grid_size
    fine_height, fine_width = grid_size
    factor_y = fine_height / coarse_height
    factor_x = fine_width / coarse_width

    if not (fine_height % coarse_height == 0 and fine_width % coarse_width == 0):
        raise ValueError("Fine grid dimensions must be divisible by coarse grid dimensions.")

    frames_per_snapshot = max(int(snapshot_interval_sec * frame_rate), 1)
    num_frames = data.shape[0]
    num_snapshots = num_frames // frames_per_snapshot

    # Extract positions as NumPy arrays
    if isinstance(data.columns, pd.MultiIndex):
        ant_indices = data.columns.get_level_values(0).unique()
        x_cols = [col for col in data.columns if col[1] == 'x']
        y_cols = [col for col in data.columns if col[1] == 'y']
        x_positions = data[x_cols].values  # Shape: (num_frames, num_ants)
        y_positions = data[y_cols].values
        num_ants = len(ant_indices)
    else:
        raise ValueError("DataFrame does not have MultiIndex columns.")

    print(f"Number of Ants: {num_ants}")
    print(f"Shape of X Positions: {x_positions.shape}")
    print(f"Shape of Y Positions: {y_positions.shape}")

    # Precompute relative indices
    bounding_box_coarse = max(1, math.ceil(bounding_box / factor_x))
    half_box = bounding_box_coarse // 2
    relative_indices = [(dy, dx) for dy in range(-half_box, half_box + 1)
                        for dx in range(-half_box, half_box + 1)]

    # **First Pass: Determine global_max**
    print("First Pass: Determining global maximum pheromone intensity...")
    pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)
    global_max = 1e-8  # Initialize with a small number to avoid division by zero

    with tqdm(total=num_frames, desc="First Pass") as pbar:
        for frame_idx in range(num_frames):
            # Apply decay
            pheromone_map *= (1 - decay_rate)

            # Deposit pheromones for all ants
            for ant in range(num_ants):
                x_fine = x_positions[frame_idx, ant]
                y_fine = y_positions[frame_idx, ant]
                if np.isnan(x_fine) or np.isnan(y_fine):
                    continue
                x_coarse = int(x_fine // factor_x)
                y_coarse = int(y_fine // factor_y)
                if x_coarse < 0 or x_coarse >= coarse_width or y_coarse < 0 or y_coarse >= coarse_height:
                    continue
                # Apply deposition
                for dy, dx in relative_indices:
                    x = x_coarse + dx
                    y = y_coarse + dy
                    if 0 <= x < coarse_width and 0 <= y < coarse_height:
                        pheromone_map[y, x] += deposition_rate

            # Update global_max
            current_max = pheromone_map.max()
            if current_max > global_max:
                global_max = current_max

            # Debug: Print current_max at intervals
            # if frame_idx % 100000 == 0:
            #     print(f"Frame {frame_idx}: Current Max = {current_max}")

            pbar.update(1)

    print(f"Global maximum pheromone intensity determined: {global_max}")

    # **Second Pass: Save scaled snapshots**
    print("Second Pass: Saving scaled pheromone snapshots...")
    pheromone_map = np.zeros(coarse_grid_size, dtype=np.float32)
    snapshots = np.zeros((num_snapshots, coarse_height, coarse_width), dtype=np.float32)
    snapshot_counter = 0

    with tqdm(total=num_frames, desc="Second Pass") as pbar:
        for frame_idx in range(num_frames):
            # Apply decay
            pheromone_map *= (1.0 - decay_rate)

            # Deposit pheromones for all ants
            for ant in range(num_ants):
                x_fine = x_positions[frame_idx, ant]
                y_fine = y_positions[frame_idx, ant]
                if np.isnan(x_fine) or np.isnan(y_fine):
                    continue
                x_coarse = int(x_fine // factor_x)
                y_coarse = int(y_fine // factor_y)
                if x_coarse < 0 or x_coarse >= coarse_width or y_coarse < 0 or y_coarse >= coarse_height:
                    continue
                # Apply deposition
                for dy, dx in relative_indices:
                    x = x_coarse + dx
                    y = y_coarse + dy
                    if 0 <= x < coarse_width and 0 <= y < coarse_height:
                        pheromone_map[y, x] += deposition_rate

            # Take snapshot if needed
            if (frame_idx + 1) % frames_per_snapshot == 0:
                snapshot = pheromone_map.copy()
                if scale:
                    snapshot /= global_max
                    snapshot = np.clip(snapshot, 0, 1)
                    # snapshot = sigmoid_normalization(snapshot, k=0.1, x0=global_max / 2)
                    # snapshot = np.clip(snapshot, 0, 1)  # Optional: Ensure values are within [0, 1]
                snapshots[snapshot_counter] = snapshot
                snapshot_counter += 1

            pbar.update(1)

    # Save all snapshots at once
    with h5py.File(output_file_path, 'w') as h5f:
        dset = h5f.create_dataset(
            "pheromone_time_series",
            data=snapshots,
            dtype='float32',
            compression="gzip"  # Optional: Compress data to save space
        )

    print(f"Pheromone time series saved to '{output_file_path}' as an HDF5 file.")


In [ ]:
output_file = "pheromone_time_series_discrete.h5"
calculate_pheromone_intensity_two_pass(
    data=data,
    grid_size=(900, 900),
    coarse_grid_size=(50, 50),  # Discretized to 20x20 grid
    bounding_box=5,             # Bounding box in coarse grid
    decay_rate=0.01,
    deposition_rate=0.1,
    scale=True,
    frame_rate=60,
    snapshot_interval_sec=1.0,  # Capture every 1 second
    save_to_file=True,
    output_file_path=os.path.join(DATA_DIRECTORY, output_file)
)

Number of Ants: 57
Shape of X Positions: (864003, 57)
Shape of Y Positions: (864003, 57)
First Pass: Determining global maximum pheromone intensity...


First Pass: 100%|██████████| 864003/864003 [01:35<00:00, 9021.86it/s]


Global maximum pheromone intensity determined: 79.99847412109375
Second Pass: Saving scaled pheromone snapshots...


Second Pass: 100%|██████████| 864003/864003 [01:34<00:00, 9122.84it/s]


Pheromone time series saved to 'data/2023_2/pheromone_time_series_discrete.h5' as an HDF5 file.
